# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846767683604                   -0.70    4.5         
  2   -7.852312670538       -2.26       -1.53    1.0   33.2ms
  3   -7.852615262759       -3.52       -2.55    1.5   35.4ms
  4   -7.852645808262       -4.52       -2.89    2.5   46.2ms
  5   -7.852646498084       -6.16       -3.21    1.0   35.4ms
  6   -7.852646677151       -6.75       -4.01    1.2   34.7ms
  7   -7.852646686569       -8.03       -5.27    2.0   42.9ms
  8   -7.852646686723       -9.81       -5.37    2.0   42.6ms
  9   -7.852646686729      -11.18       -6.07    1.0   34.5ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846848862373                   -0.70           4.8         
  2   -7.852527583778       -2.25       -1.64   0.80    2.0    320ms
  3   -7.852636706607       -3.96       -2.72   0.80    1.0   33.1ms
  4   -7.852646469502       -5.01       -3.28   0.80    2.2   39.3ms
  5   -7.852646674862       -6.69       -4.14   0.80    1.2   31.1ms
  6   -7.852646686498       -7.93       -4.82   0.80    1.8   38.2ms
  7   -7.852646686724       -9.65       -5.54   0.80    1.8   36.5ms
  8   -7.852646686729      -11.28       -6.95   0.80    1.8   35.7ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.361254e+01     3.173737e+00
 * time: 0.4934232234954834
     1     1.236214e+00     1.883770e+00
 * time: 0.7537941932678223
     2    -1.576311e+00     2.261677e+00
 * time: 0.7892501354217529
     3    -3.801915e+00     2.028634e+00
 * time: 0.841076135635376
     4    -5.070024e+00     1.883327e+00
 * time: 0.8906559944152832
     5    -6.721066e+00     1.057799e+00
 * time: 0.9402980804443359
     6    -7.370043e+00     7.923213e-01
 * time: 0.9921681880950928
     7    -7.585448e+00     6.986792e-01
 * time: 1.025482177734375
     8    -7.721677e+00     2.260344e-01
 * time: 1.0599720478057861
     9    -7.779555e+00     7.979569e-02
 * time: 1.0936391353607178
    10    -7.820037e+00     1.286127e-01
 * time: 1.1284830570220947
    11    -7.833926e+00     4.920357e-02
 * time: 1.165215015411377
    12    -7.846283e+00     2.430614e-02
 * time: 1.2030410766601562
    13    -7.850648e+00     1.896460e-02
 * time: 1.2385110855102

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846857445532                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645916097                   -1.64         
  2   -7.852646686730       -6.11       -3.71    2.23s
  3   -7.852646686730      -13.35       -7.25    234ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 5.734891767279591e-7
|ρ_newton - ρ_scfv| = 3.9568421951431614e-7
|ρ_newton - ρ_dm|   = 7.414466217838601e-10
